# Looking at https://github.com/ytsvetko/metaphor

In [153]:
##all the imports...
#%pylab inline
import re
import math
import string
import nltk
import pprint
import matplotlib
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
sent_tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
from collections import Counter
from __future__ import division
from nltk.collocations import *
import string, random
from nltk.corpus import brown
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk import bigrams
from nltk import collocations
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.corpus import brown

## Start by loading the .csv files and setting the columns.
### For these .csv files, downloaded from https://github.com/ytsvetko/metaphor inputs and saved as .csv with the first row added as "sample"

In [33]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet.head()

,sample
0,angry welt
1,bald assertion
2,bare outline
3,black humor
4,blind alley


In [34]:
df_anmet = pd.read_csv("an_mets.csv", low_memory=False)
df_anmet['metaphor'] = 1
df_anmet['an'] = 1
df_anmet['svo'] = 0
df_anmet['metanet'] = 0
df_anmet.head()
len(df_anmet)

100

In [35]:
df_annonmet = pd.read_csv("an_nonmets.csv", low_memory=False)
df_annonmet['metaphor'] = 0
df_annonmet['an'] = 1
df_annonmet['svo'] = 0
df_annonmet['metanet'] = 0
df_annonmet.head()
len(df_annonmet)

100

In [47]:
df_svomet = pd.read_csv("svo_mets.csv", low_memory=False)
df_svomet['metaphor'] = 1
df_svomet['an'] = 0
df_svomet['svo'] = 1
df_svomet['metanet'] = 0
df_svomet.head()
len(df_svomet)

111

In [48]:
df_svononmet = pd.read_csv("svo_nonmets.csv", low_memory=False)
df_svononmet['metaphor'] = 0
df_svononmet['an'] = 0
df_svononmet['svo'] = 1
df_svononmet['metanet'] = 0
df_svononmet.head()
len(df_svononmet)

111

In [49]:
df_metanet = pd.read_csv("metanet.csv", low_memory=False)
df_metanet['metaphor'] = 1
df_metanet['an'] = 0
df_metanet['svo'] = 0
df_metanet['metanet'] = 1
df_metanet.head()
# len(df_metanet)

,sample,metaphor,an,svo,metanet
0,ability to evaluate government is ability to see,1,0,0,1
1,ability to evaluate is ability to see,1,0,0,1
2,ability to know is ability to see,1,0,0,1
3,abusive political leaders are physical bullies,1,0,0,1
4,accepting is swallowing,1,0,0,1


In [66]:
#combine into one df
frames = [df_anmet, df_annonmet, df_svomet, df_svononmet, df_metanet]
df_combo = pd.concat(frames)
df_combo.reset_index(drop=True, inplace=True)
df_combo.shape
# df_combo.replace(to_replace='none', value="")
# #         tokenizer = RegexpTokenizer(r'\w+')
# #         word_list = tokenizer.tokenize(line)
# #         filtered_words = [word for word in word_list if word not in stopwords.words('english')]
# df_combo.replace(to_replace='is', value="")
stop = ['*none', '.', 'is']
df_combo['sample'].apply(lambda x: [item for item in x if item not in stop])
df_combo[200:500]

,sample,metaphor,an,svo,metanet
200,conversation turn subject,1,0,1,0
201,resumption bring relief,1,0,1,0
202,economy move direction,1,0,1,0
203,service meet expectation,1,0,1,0
204,material live dream,1,0,1,0
205,unemployment stand *none,1,0,1,0
206,action talk *none,1,0,1,0
207,statement sit *none,1,0,1,0
208,income fall *none,1,0,1,0
209,silence speak volume,1,0,1,0


## Break down into training, dev, and test sets

In [68]:
random_index = np.random.permutation(df_combo.index)
df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled = df_combo.ix[random_index, ['sample', 'metaphor', 'an', 'svo', 'metanet']]
df_shuffled.reset_index(drop=True, inplace=True)
len(df_shuffled)

1606

In [69]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)
#train_size = round(rows*.6)
train_size = round(rows*.9)
#dev_size   = round(rows*.2)
dev_size   = round(rows*.1)
df_train = df_shuffled.loc[:train_size]
df_train.shape
df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
df_dev.shape
df_test = df_shuffled.loc[dev_size+train_size:].reset_index(drop=True)
df_test.shape

Rows: 1606
Columns: 5


(0, 5)

## CountVectorizer
### Use count vecotrizer with df = 3 for unigram and bigrams to predict dev set

In [28]:
vocab = list(set(brown.words()))

In [70]:
# vec = CountVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b', analyzer=u'word', min_df=1, vocabulary=vocab)
# df_train = df_train.fillna("")
# df_dev = df_dev.fillna("")
# df_test = df_test.fillna("")


vec = TfidfVectorizer(ngram_range=(2, 3), token_pattern=r'\b\w+\b', analyzer=u'char', min_df=1)
df_train = df_train.fillna("")
df_dev = df_dev.fillna("")
df_test = df_test.fillna("")

In [71]:
arr_train_feature_sparse = vec.fit_transform(df_train['sample'])
arr_train_feature_sparse
arr_train_feature = arr_train_feature_sparse.toarray()
feature_labels = vec.get_feature_names()

In [72]:
arr_dev_feature_sparse = vec.transform(df_dev["sample"])
arr_dev_feature = arr_dev_feature_sparse.toarray()

In [73]:
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor']) #defining features (from reviews) and passing in Category label
logreg_predictions = logreg_model.predict(arr_dev_feature)
accuracy_score(df_dev['metaphor'], logreg_predictions)

0.91925465838509313

In [74]:
feature_sum = arr_train_feature.sum(axis=0)   #sum the counts of each feature

df_feature_sum = pd.DataFrame({'counts': feature_sum})
df_feature_sum.index = vec.get_feature_names()
df_feature_sum.sort('counts', ascending=False)[0:10]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,counts
s,56.150914
in,53.809020
i,51.863619
is,50.957381
a,49.962174
ti,49.516231
on,49.403087
is,44.935289
is,44.639800
re,41.980039


In [322]:
## make a file of sample sentences to practice on
## loop through sentence and .split()
## use nltk bigrams
##
## put into pandas as df_test

In [75]:
word_list = 'Jamal was a pig at dinner'.split()
filtered_words = [word for word in word_list if word not in stopwords.words('english')]
filtered_words
testmeta = list(trigrams(filtered_words))
testmeta
testmeta1 = [' '.join(x) for x in testmeta]
testmeta1
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample']
print(filtered_words)

['Jamal', 'pig', 'dinner']


In [76]:
arr_test_feature_sparse = vec.transform(df_test['sample'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
logreg_predictions

array([1])

In [24]:
df_test = pd.DataFrame(testmeta1)
df_test

,0
0,Jamal pig dinner


In [77]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
#         word_list = line.split()
        tokenizer = RegexpTokenizer(r'\w+')
        word_list = tokenizer.tokenize(line)
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
#         testmeta1.append(' '.join(filtered_words))
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
#         testmeta1.append(' '.join(x) for x in filtered_words)
# print(testmeta1)

df_test =pd.DataFrame(testmeta1)
df_test
df_test = df_test.fillna("")
df_test
# df_test.columns = ['sample']
# df_test = pd.DataFrame(testmeta1)
# df_test
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
# df_test = df_test.fillna("")
# df_test

In [78]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor0'] + df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] + df_test['metaphor6'] 

def count_metaphors():
    if sum(df_test['sum_metaphor']) >= 1:
        df_test['sum_metaphor'] = 1
    else:
        df_test['sum_metaphor'] = 0
count_metaphors()
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6,metaphor0,metaphor1,metaphor2,metaphor3,metaphor4,metaphor5,metaphor6,sum_metaphor
0,The snow,snow white,white blanket,,,,,1,1,1,1,1,1,1,1
1,The hospital,hospital refrigerator,,,,,,1,1,1,1,1,1,1,1
2,The classroom,classroom zoo,,,,,,1,1,1,1,1,1,1,1
3,America melting,melting pot,,,,,,1,1,1,1,1,1,1,1
4,Her lovely,lovely voice,voice music,music ears,,,,1,1,1,1,1,1,1,1
5,Life roller,roller coaster,,,,,,1,1,1,1,1,1,1,1
6,The alligator,alligator teeth,teeth white,white daggers,,,,1,1,1,1,1,1,1,1
7,Their home,home prison,,,,,,1,1,1,1,1,1,1,1
8,The slide,slide playground,playground hot,hot stove,,,,1,1,1,1,1,1,1,1
9,His heart,heart cold,cold iron,,,,,1,1,1,1,1,1,1,1


In [280]:
with open("an_nonmets.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in bigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test
df_test.columns = ['sample0']
df_test = df_test.fillna("")
df_test

arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

sum(df_test['metaphor0'])
#len(df_test['metaphor0'])

3

In [79]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = [word for word in word_list if word not in stopwords.words('english')]
        testmeta1.append(' '.join(x) for x in trigrams(filtered_words))
df_test = pd.DataFrame(testmeta1)
df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0', 'sample1', 'sample2', 'sample3', 'sample4', 'sample5', 'sample6']
df_test = df_test.fillna("")
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;
7,All light networks.,,,,,,
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas"
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,


In [80]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample1'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor1'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample2'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor2'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample3'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor3'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample4'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor4'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample5'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor5'] = logreg_predictions

arr_test_feature_sparse = vec.transform(df_test['sample6'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor6'] = logreg_predictions

df_test['sum_metaphor'] = df_test['metaphor1'] + df_test['metaphor2'] + df_test['metaphor3'] +df_test['metaphor4'] +df_test['metaphor5'] 

if sum(df_test['sum_metaphor']) >= 1:
    df_test['sum_metaphor'] = 1
sum(df_test['sum_metaphor'])
#len(df_test['sum_metaphor'])
df_test

,sample0,sample1,sample2,sample3,sample4,sample5,sample6,metaphor0,metaphor1,metaphor2,metaphor3,metaphor4,metaphor5,metaphor6,sum_metaphor
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words,meanings words arranged,words arranged across,1,1,1,1,1,1,1,1
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays",atlas displays rainbow,displays rainbow hues,1,1,1,1,1,1,1,1
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one,shows one specific,one specific aspect,1,1,1,1,1,1,1,1
3,No single brain,single brain region,brain region holds,region holds one,holds one word,one word concept.,,1,1,1,1,1,1,1,1
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related,number related words.,,1,1,1,1,1,1,1,1
5,And single word,single word lights,word lights many,lights many different,many different brain,different brain spots.,,1,1,1,1,1,1,1,1
6,Together make networks,make networks represent,networks represent meanings,represent meanings word,meanings word use:,word use: life,use: life love;,1,1,1,1,1,1,1,1
7,All light networks.,,,,,,,1,1,1,1,1,1,1,1
8,Described “tour de,“tour de force”,de force” one,force” one researcher,one researcher involved,"researcher involved study,","involved study, atlas",1,1,1,1,1,1,1,1
9,"With advances, technology","advances, technology could",technology could profound,could profound impact,profound impact medicine,impact medicine fields.,,1,1,1,1,1,1,1,1


In [101]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [103]:
arr_test_feature_sparse = vec.transform(df_test['sample0'])
arr_test_feature = arr_test_feature_sparse.toarray()
logreg = LogisticRegression()
logreg_model = logreg.fit(arr_train_feature, df_train['metaphor'])
logreg_predictions = logreg_model.predict(arr_test_feature)
indices = [i for i, x in enumerate(logreg_predictions) if x == 0]
df_test['metaphor0'] = logreg_predictions

df_test

,sample0,metaphor0
0,scientists created “atlas brain” reveals meani...,1
1,"like colourful quilt laid cortex, atlas displa...",1
2,“our goal build giant atlas shows one specific...,1
3,no single brain region holds one word concept.,1
4,a single brain spot associated number related ...,1
5,and single word lights many different brain sp...,1
6,together make networks represent meanings word...,1
7,all light networks.,1
8,described “tour de force” one researcher invol...,1
9,"with advances, technology could profound impac...",1


In [81]:
df_test.ix[:, 0:5].head()

,sample0,sample1,sample2,sample3,sample4
0,Scientists created “atlas,created “atlas brain”,“atlas brain” reveals,brain” reveals meanings,reveals meanings words
1,Like colourful quilt,colourful quilt laid,"quilt laid cortex,","laid cortex, atlas","cortex, atlas displays"
2,“Our goal build,goal build giant,build giant atlas,giant atlas shows,atlas shows one
3,No single brain,single brain region,brain region holds,region holds one,holds one word
4,A single brain,single brain spot,brain spot associated,spot associated number,associated number related


In [179]:
len(df_test.columns)
#df_test[0]

6

In [ ]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    for line in testset:
        word_list = line.split()
        filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        testmeta1.append(filtered_words)
df_test = pd.DataFrame(testmeta1)
df_test
# df_test = df_test.ix[:, 0:6]
df_test.columns = ['sample0']
df_test = df_test.fillna("")
# df_test

In [104]:
import nltk, re
from nltk.corpus import brown
from nltk import word_tokenize

In [105]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    return t2


def train_tagger(already_tagged_sents):
    train_sents, test_sents = create_data_sets(already_tagged_sents)
    ngram_tagger = build_backoff_tagger(train_sents)
    print ("%0.3f pos accuracy on test set" % ngram_tagger.evaluate(test_sents))
    return ngram_tagger


In [106]:
def train_tagger_on_brown():
    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance','science_fiction'])
    return train_tagger(brown_tagged_sents)


In [107]:
def tokenize_text(corpus):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences    
    return [nltk.word_tokenize(word) for word in raw_sents]

def create_corpus(f):
    with open(f, 'r') as text_file:
        new_corpus = text_file.read()
    return new_corpus


In [108]:
brown_tagger = train_tagger_on_brown()

0.911 pos accuracy on test set


In [176]:
with open("sciencearticle_line.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

[[('scientists', 'NNS'),
  ('have', 'HV'),
  ('created', 'VBN'),
  ('an', 'AT'),
  ('atlas', 'NN'),
  ('of', 'IN'),
  ('the', 'AT'),
  ('brain', 'NN'),
  ('that', 'CS'),
  ('reveals', 'VBZ'),
  ('how', 'WRB'),
  ('the', 'AT'),
  ('meanings', 'NNS'),
  ('of', 'IN'),
  ('words', 'NNS'),
  ('are', 'BER'),
  ('arranged', 'VBN'),
  ('across', 'IN'),
  ('different', 'JJ'),
  ('regions', 'NNS'),
  ('of', 'IN'),
  ('the', 'AT'),
  ('organ', 'NN')],
 [('like', 'CS'),
  ('a', 'AT'),
  ('colourful', 'NN'),
  ('quilt', 'NN'),
  ('laid', 'VBN'),
  ('over', 'IN'),
  ('the', 'AT'),
  ('cortex', 'NN'),
  ('the', 'AT'),
  ('atlas', 'NN'),
  ('displays', 'VBZ'),
  ('in', 'IN'),
  ('rainbow', 'NN'),
  ('hues', 'NNS'),
  ('how', 'WRB'),
  ('individual', 'JJ'),
  ('words', 'NNS'),
  ('and', 'CC'),
  ('the', 'AT'),
  ('concepts', 'NNS'),
  ('they', 'PPSS'),
  ('convey', 'VB'),
  ('can', 'MD'),
  ('be', 'BE'),
  ('grouped', 'VBN'),
  ('together', 'RB'),
  ('in', 'IN'),
  ('clumps', 'NNS'),
  ('of', 'IN'),
  

In [177]:
def train_tagger_on_brown_augmented_with_additional_sents():

    additional_sents = [[('colorful', 'JJ'), ('quilt', 'NN')],
                        [('regions', 'NN'), ('represent', 'VB'), ('world', 'NN')],
                        [('public', 'NN'), ('explore', 'VB'), ('brain', 'NN')]]


    brown_tagged_sents = brown.tagged_sents(categories=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
    'humor', 'learned', 'lore', 'mystery', 'religion', 'reviews', 'romance', 'science_fiction'])
    
    #append hand-tagged cooking sentences to the front of the training data
    all_tagged_sents = additional_sents + brown_tagged_sents
    return train_tagger(all_tagged_sents)

brown_and_additional_tagger = train_tagger_on_brown_augmented_with_cooking_sents()

0.911 pos accuracy on test set


In [173]:
better_sentences = get_pos(testmeta1, brown_and_additional_tagger)

In [178]:
with open("test_meta_sent.txt", "r") as testset:
    filtered_words = []
    testmeta = []
    testmeta1 = []
    word_list = ''
    tokenizer = RegexpTokenizer(r'\w+')
    for line in testset:
        word_list = line.split()
#         filtered_words = ' '.join([word.lower() for word in word_list if word not in stopwords.words('english')])
        filtered_words = ' '.join([word.lower() for word in word_list])
        testmeta1.append(tokenizer.tokenize(filtered_words))
testmeta1[0]

def get_pos(sents, tagger):        
    return [tagger.tag(sent) for sent in sents]

pos = get_pos(testmeta1, brown_tagger)
pos

[[('the', 'AT'),
  ('snow', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('white', 'JJ'),
  ('blanket', 'NN')],
 [('the', 'AT'),
  ('hospital', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('refrigerator', 'NN')],
 [('the', 'AT'),
  ('classroom', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('zoo', 'NN')],
 [('america', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('melting', 'VBG'),
  ('pot', 'NN')],
 [('her', 'PP$'),
  ('lovely', 'JJ'),
  ('voice', 'NN'),
  ('was', 'BEDZ'),
  ('music', 'NN'),
  ('to', 'IN'),
  ('his', 'PP$'),
  ('ears', 'NNS')],
 [('life', 'NN'),
  ('is', 'BEZ'),
  ('a', 'AT'),
  ('roller', 'NN'),
  ('coaster', 'NN')],
 [('the', 'AT'),
  ('alligator', 'NN'),
  ('s', 'NN'),
  ('teeth', 'NNS'),
  ('are', 'BER'),
  ('white', 'JJ'),
  ('daggers', 'NN')],
 [('their', 'PP$'),
  ('home', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('prison', 'NN')],
 [('the', 'AT'),
  ('slide', 'NN'),
  ('on', 'IN'),
  ('the', 'AT'),
  ('playground', 'NN'),
  ('was', 'BEDZ'),
  ('a', 'AT'),
  ('hot', 'J